# [Step 3] นำข้อมูลภาพถ่ายดาวเทียม Sentinel - 2 ที่ทำการ Pre-processing มาจำแนก

In [1]:
import os
import pyrsgis.raster as raster
import tensorflow as tf
import tensorflow.keras as ks

In [ ]:
#ตั้งค่า Directory ในการนำเข้าตัว Model และข้อมูลที่ต้องการใช้ในกา่รจำแนก
os.chdir(r"C:\Users\std\Desktop\Dataset\CNN Method\SplitIMG_3x3\Output")
#import Model
Current_Model = ks.models.load_model('LandUse_Model/CNN_LU_3x3_HKT.h5')

### [Step 3.1] นำเข้าข้อมูลภาพถ่ายดาวเทียมที่ได้เตรียมไว้ก่อนหน้านี้

In [ ]:
os.chdir(r"C:\Users\std\Desktop\Dataset\CNN Method")

#ทำการ Read File ภาพถ่ายดาวเทียม
HKT_IMG = "Feature_HKT_Inner_DOS_09022018_in_255.tif"
ds, HKT_IMG = raster.read(HKT_IMG, bands='all')

### [Step 3.2] สร้าง Function Generator เพื่อทำการตัดภาพให้มีขนาด 5x5 px (เท่ากับตอนที่ Train model)

In [2]:
#ฟังก์ชั่นตัวนี้ใช้ สามารถจำนวนภาพขนาดเท่าไหร่ก็ได้ ในที่นี้แนะนำให้ทำขนาด kSize = 5
def CNNdataGenerator(mxBands, kSize):
    mxBands = mxBands / 255.0     #normalize
    nBands, rows, cols = mxBands.shape
    margin = math.floor(kSize/2)
    mxBands = np.pad(mxBands, margin, mode='constant')[margin:-margin, :, :]

    features = np.empty((rows*cols, kSize, kSize, nBands))

    n = 0
    for row in range(margin, rows+margin):
        for col in range(margin, cols+margin):
            feat = mxBands[:, row-margin:row+margin+1, col-margin:col+margin+1]

            #b1, b2, b3, b4, b5, b6 = feat
            #feat = np.dstack((b1, b2, b3, b4, b5, b6))
            b1, b2, b3, b4 = feat
            feat = np.dstack((b1, b2, b3, b4))
            features[n, :, :, :] = feat
            n += 1
            
    return(features)

### [Step 3.3] ขั้นตอนสุดท้าย !!! สำหรับการนำภาพถ่ายที่เตรียมไว้ไปจำแนกด้วยตัว CNN Model และได้ output ออกมา

In [ ]:
#เรียกใช้งานฟังก์ชั่น CNNdataGenerator
IMG_Promp = CNNdataGenerator(HKT_IMG, kSize=5)
print(f"Data Shape : {IMG_Promp.shape}")

#ทำการนำเอาข้อมูลไปทำการจำแนกและ Export ข้อมูลออกมาเมื่อเสร็จ
new_HKT = Current_Model.predict(IMG_Promp)
newPredicted = new_HKT[:,1]

prediction = np.reshape(newPredicted, (ds.RasterYSize, ds.RasterXSize))
outFile = 'HKT_CNN_predicted_5by5_09_02_2018.tif'  #สามารถกำหนดชื่อ output ได้ตามต้องการ
raster.export(prediction, ds, filename=outFile, dtype='int')